In [37]:
!nvidia-smi

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Tue Feb 16 06:32:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    28W /  70W |   1634MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+-------------------

In [11]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
import os
os.environ['KAGGLE_CONFIG_DIR']='/content/gdrive/MyDrive/kaggle_datasets'

In [14]:
%cd '/content/gdrive/MyDrive/kaggle_datasets'

/content/gdrive/MyDrive/kaggle_datasets


In [15]:
!ls

cat-and-dog
checkpoint
colab_fruits.h5
colab_fruits_weights.data-00000-of-00001
colab_fruits_weights.index
colab_inception_resnet_malaria_detection_model.h5
covid19-image-dataset
fruits-fresh-and-rotten-for-classification
kaggle.json
messy-vs-clean-room
model-for-detecting-malaria
natural-images
real-and-fake-face-detection


In [16]:
!kaggle datasets download -d l33tc0d3r/indian-food-classification

 99% 1.47G/1.48G [00:10<00:00, 133MB/s]
100% 1.48G/1.48G [00:10<00:00, 147MB/s]


In [17]:
!mkdir indian-food-classification
!mv indian-food-classification.zip indian-food-classification

In [18]:
%cd indian-food-classification

/content/gdrive/My Drive/kaggle_datasets/indian-food-classification


In [19]:
!unzip indian-food-classification.zip

Streaming output truncated to the last 5000 lines.
  inflating: Food Classification/chapati/253.jpg  
  inflating: Food Classification/chapati/254.jpg  
  inflating: Food Classification/chapati/255.jpg  
  inflating: Food Classification/chapati/256.jpg  
  inflating: Food Classification/chapati/257.jpg  
  inflating: Food Classification/chapati/258.jpg  
  inflating: Food Classification/chapati/259.jpg  
  inflating: Food Classification/chapati/260.jpg  
  inflating: Food Classification/chapati/261.jpg  
  inflating: Food Classification/chapati/262.jpg  
  inflating: Food Classification/chapati/263.jpg  
  inflating: Food Classification/chapati/264.jpg  
  inflating: Food Classification/chapati/265.jpg  
  inflating: Food Classification/chapati/266.jpg  
  inflating: Food Classification/chapati/267.jpg  
  inflating: Food Classification/chapati/268.jpg  
  inflating: Food Classification/chapati/269.jpg  
  inflating: Food Classification/chapati/270.jpg  
  inflating: Food Classificatio

In [20]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [21]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2

In [22]:
conv_base = ResNet50V2(include_top=False, input_shape=(150,150,3))

94674944/94668760 [==============================] - 1s 0us/step


In [23]:
conv_base.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 77, 77, 64)   0           conv1_conv[0][0]                 
_________________________________________________________________________________________

In [24]:
for layer in conv_base.layers:
  layer.trainable = False

In [26]:
train_dir = '/content/gdrive/MyDrive/kaggle_datasets/indian-food-classification/Food Classification'
val_dir = '/content/gdrive/MyDrive/kaggle_datasets/indian-food-classification/Food Classification'

In [27]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.3)

val_datagen = ImageDataGenerator(rescale=1./255)

In [28]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150,150),
                                                    batch_size=16,
                                                    class_mode='categorical',
                                                    subset='training')

validation_generator = train_datagen.flow_from_directory(val_dir,
                                                        target_size=(150,150),
                                                        batch_size=16,
                                                        class_mode='categorical',
                                                        subset='validation')

Found 4399 images belonging to 20 classes.
Found 1870 images belonging to 20 classes.


In [29]:
from tensorflow.keras import models
from tensorflow.keras import layers

In [30]:
model = models.Sequential([
                           conv_base,
                           layers.Flatten(),
                           layers.Dropout((0.5)),
                           layers.Dense(512, activation='relu'),
                           layers.BatchNormalization(),
                           layers.Dense(256, activation='relu'),
                           layers.Dense(20, activation='softmax')
])

In [23]:
# model = models.Sequential([
#                            layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(150,150,3)),
#                            layers.BatchNormalization(),
#                            layers.MaxPooling2D((2,2)),
#                            layers.Dropout((0.4)),


#                            layers.Conv2D(32, (3,3), activation='relu'),
#                            layers.Conv2D(64, (3,3), activation='relu'),
#                            layers.BatchNormalization(),
#                            layers.MaxPooling2D((2,2)),
#                            layers.Dropout((0.4)),

#                            layers.Conv2D(64, (3,3), activation='relu'),
#                            layers.Conv2D(128, (3,3), activation='relu'),
#                            layers.BatchNormalization(),
#                            layers.MaxPooling2D((2,2)),
#                            layers.Dropout((0.4)),

#                            layers.Flatten(),

#                            layers.Dense(256, activation='relu'),
#                            layers.BatchNormalization(),
#                            layers.Dense(512, activation='relu'),
#                            layers.Dense(20, activation='softmax')

# ])

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 5, 5, 2048)        23564800  
_________________________________________________________________
flatten (Flatten)            (None, 51200)             0         
_________________________________________________________________
dropout (Dropout)            (None, 51200)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               26214912  
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 20)                5

In [32]:
model.compile(tf.keras.optimizers.Adam(learning_rate=2e-4),
              tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [ ]:
history = model.fit_generator(train_generator,
                             steps_per_epoch=275,
                             epochs=10,
                             validation_data=validation_generator,
                            validation_steps=117)

model.save('colab_indian_food_classification.h5')